# What

This package extracts straightforward medical rationale from Python code.  When
you decorate a function with `@trace`, the function gets compiled into a new
function. This function has instrumentation that tracks the operations it
performs on objects of type `Traceable`. After you call this function, you can
inspect the log of operations it performed on these objects. The log looks a lot
like a path through a flowchart.  Later, when you want to undersatand how the
function actually reached its result, you can query this log, pretty-print it,
etc.

In [1]:
from typing import NamedTuple
import tracer

# A very simple example

In [2]:
class User(NamedTuple):
    metric1: tracer.Float
    metric2: tracer.Float
    metric3: tracer.Int


@tracer.trace
def make_simplest_decision(user: User) -> bool:
    return user.metric3 > 5

The function returns the same plain value as it always would without tracing:

In [3]:
make_simplest_decision(User(metric1=0.6, metric2=0.6, metric3=6))

True

But we can also ask for a log of what it computed:

In [4]:
print(make_simplest_decision.trace.pretty_print())

return (metric3 > 5)


This isn't very interesting because it looks like it just printed the body fo the function.
Let's try a more complicated function:

In [5]:
@tracer.trace
def make_slightly_more_complicated_decision(user: User) -> bool:
    return user.metric1 > 0.3 and user.metric2 < 0.4 and user.metric3 > 5


make_slightly_more_complicated_decision(User(metric1=0.6, metric2=0.6, metric3=6))

False

In [6]:
print(make_slightly_more_complicated_decision.trace.pretty_print())

if (metric1 > 0.3000) (=True):
  if (metric2 < 0.4000) (=False):
    return (metric2 < 0.4000)


Notice that the compiler converted the sequence of `and` operations into nested if-statements. That's because like in C, Python's boolean operations use short-circuition.

# Functions and condition elision

We can trace operations through a complex call stack. Below, the function `make_decision_hierarchically` calls
some subroutines, some of which operate on Traceables, and some of which do not:

In [7]:
def _compute_weighted_score(user: User) -> float:
    return user.metric1 * 0.7 + user.metric2 * 0.3


def _is_safe_to_proceed(score: float) -> bool:
    if score < 0.2:
        return False
    return True


def _check_system_status() -> bool:
    # Condition that does not depend on User
    maintenance_mode = False
    if maintenance_mode:
        return False
    return True


@tracer.trace
def make_decision_hierarchically(user: User) -> bool:
    if not _check_system_status():
        return False

    weighted_score = _compute_weighted_score(user)

    if not _is_safe_to_proceed(weighted_score):
        return False

    my_threshold = 0.4 * 2
    if weighted_score > my_threshold:
        return True

    # Cascade another check dependent on original values
    if user.metric3 > 5:
        return True

    return False


make_decision_hierarchically(User(metric1=0.9, metric2=0.9, metric3=1))

True

In [8]:
print(make_decision_hierarchically.trace.pretty_print())

if (((metric1 * 0.7000) + (metric2 * 0.3000)) < 0.2000) (=False):
  if (((metric1 * 0.7000) + (metric2 * 0.3000)) > 0.8000) (=True):
    return True


There are a few important things to notice:
1. All the subroutines have been inlined in the trace. This makes it easier to just follow the flow.
2. Conditional statements that don't depend on `Traceable` objects are elided from the log. None of the fuss of `_check_system_status` appears in the log.
3. Similarly, aritmetic operations that don't depend on Traceables have been folded into a constant. Notably, `my_threshold` doesn't appear anywhere in the trace because it doesn't depend on a `Traceable`.

# The example from the README

You don't need to encapsulate arguments inside a NamedTuple. Also, you don't
need to correctly specifcy that arguments to subroutines are subtypes of
Traceable. The compiler can figure this out:

In [9]:
def calculate_risk_score(blood_pressure: float, heart_rate: int) -> float:
    """Calculate a risk score based on blood pressure and heart rate."""
    pressure_factor = blood_pressure / 100.0
    rate_factor = heart_rate / 80.0
    return pressure_factor * rate_factor


def is_critical_temperature(temperature: float) -> bool:
    """Check if temperature indicates a critical condition."""
    return temperature > 37.5


@tracer.trace
def assess_patient(
    blood_pressure: tracer.Float, heart_rate: tracer.Int, temperature: tracer.Float
) -> bool:
    bp, hr, temp = blood_pressure, heart_rate, temperature
    risk_score = calculate_risk_score(bp, hr)

    if risk_score > 1.5:
        if hr > 100:
            return is_critical_temperature(temp)
    return False


assess_patient(130.0, 110, 38.0)

True

In [10]:
print(assess_patient.trace.pretty_print())

if (((blood_pressure / 100.0000) * (heart_rate / 80.0000)) > 1.5000) (=True):
  if (heart_rate > 100) (=True):
    return (temperature > 37.5000)


Notice how we were able to trace through `calculate_risk_factor` despite the
type annotations.  Also notice how despite the Traceable objects being renamed
to `bp`, `hr`, and `temp`, the rationale expression can still retrieve their
original names.

# Examples that might be hard for runtime tracing

In [11]:
@tracer.trace
def useless_traceable(traceable: tracer.Float) -> float:
    if traceable > 0.5:
        a = 2
    else:
        a = 0.5

    return 3

useless_traceable(0.6)

3

In [12]:
print(useless_traceable.trace.pretty_print())

return 3


Notice how the compiler is able to eliminate the spurious conditional.

In [13]:
@tracer.trace
def useless_traceable2(traceable: tracer.Float, traceable2: tracer.Float) -> float:
    r = (traceable > 0.5)

    if traceable2 > 0:
        return r
    else:
        return -1

useless_traceable2(0.6, 0.6)

1

In [14]:
print(useless_traceable2.trace.pretty_print())

if (traceable2 > 0) (=True):
  return (traceable > 0.5000)


This is good! It's not getting confused by `r = (traceable > 0.5)` before the if-statement.